In [15]:
import os
import time
from random import randint
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import nibabel as nib
import nilearn as nl
import nilearn.plotting as nlplt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from torch.optim import Adamax
from torch.nn import MSELoss


from the_rest import *

import warnings
warnings.simplefilter("ignore")

In [56]:
model = GSNet(img_ch=4, output_ch=3).to('cuda')

In [ ]:
!pip install torchsummaryX
from torchsummaryX import summary

In [58]:
model = model.cuda()
input_tensor=torch.zeros((1, 4, 128, 128, 128)).cuda()
sum = summary(model, input_tensor)

                                            Kernel Shape  \
Layer                                                      
0_Conv1.conv2.Conv3d_0                  [4, 16, 3, 3, 3]   
1_Conv1.conv2.InstanceNorm3d_1                         -   
2_Conv1.conv2.Dropout3d_2                              -   
3_Conv1.conv2.ReLU_3                                   -   
4_Conv1.conv2.Conv3d_4                 [16, 16, 3, 3, 3]   
5_Conv1.conv2.InstanceNorm3d_5                         -   
6_Conv1.conv2.Dropout3d_6                              -   
7_Conv1.conv2.ReLU_7                                   -   
8_Maxpool                                              -   
9_Conv2.conv2.Conv3d_0                 [16, 32, 3, 3, 3]   
10_Conv2.conv2.InstanceNorm3d_1                        -   
11_Conv2.conv2.Dropout3d_2                             -   
12_Conv2.conv2.ReLU_3                                  -   
13_Conv2.conv2.Conv3d_4                [32, 32, 3, 3, 3]   
14_Conv2.conv2.InstanceNorm3d_5         

In [55]:
pd.set_option('display.max_rows', None)
sum

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_Conv1.conv2.Conv3d_0,"[4, 16, 3, 3, 3]","[1, 16, 128, 128, 128]",1728.0,3.623879e+09
1_Conv1.conv2.InstanceNorm3d_1,-,"[1, 16, 128, 128, 128]",NaN,NaN
2_Conv1.conv2.Dropout3d_2,-,"[1, 16, 128, 128, 128]",NaN,NaN
3_Conv1.conv2.ReLU_3,-,"[1, 16, 128, 128, 128]",NaN,NaN
4_Conv1.conv2.Conv3d_4,"[16, 16, 3, 3, 3]","[1, 16, 128, 128, 128]",6912.0,1.449551e+10
5_Conv1.conv2.InstanceNorm3d_5,-,"[1, 16, 128, 128, 128]",NaN,NaN
6_Conv1.conv2.Dropout3d_6,-,"[1, 16, 128, 128, 128]",NaN,NaN
7_Conv1.conv2.ReLU_7,-,"[1, 16, 128, 128, 128]",NaN,NaN
8_Maxpool,-,"[1, 16, 64, 64, 64]",NaN,NaN



---
|                 | Totals            |
|-----------------|------------------|
| Total params    | 6.493291M        |
| Trainable params| 6.493291M        |
| Non-trainable params | 0.0          |
| Mult-Adds       | 175.528738816G   |
---